In [ ]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
%run helper.ipynb

In [3]:
df = pd.read_csv("data/training_data.tsv.gz", sep="\t", header=None).head(10000)
df.rename(columns={0: 'index', 1: 'title', 2: 'text', 3: 'labels'}, inplace=True)
df.drop('index', axis=1, inplace=True)

In [4]:
# Get the list of stopwords
nltk.download('stopwords')
stopwords_list = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andreas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
X = df.drop('labels', axis=1)
X = X.applymap(lambda x: re.sub(r'<.*?>|[^\w\s]', '', x.lower())).applymap(lambda x: ' '.join([word for word in x.split() if word not in stopwords_list]))
y = df['labels'].str.get_dummies(',')

X  = X["title"] + X["text"]

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [26]:
# Vectorize the corpus using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

# Vectorize the new post
X_test_vectorized = vectorizer.transform(X_test)

# Calculate cosine similarity
similarity_scores = cosine_similarity(X_test_vectorized, X_train_vectorized)

# Find the closest label for each new post
y_pred = []
for i in range(len(X_test)):
    closest_label_index = similarity_scores[i].argmax()
    closest_label = y_train.loc[closest_label_index, :]
    y_pred.append(closest_label)

In [39]:
custom_f1_score(y_test, y_pred)

0.733375